In [1]:
import openai
import json
import langchain

import pandas as pd
from openai import OpenAI

from langchain_openai import OpenAI, ChatOpenAI

import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
key=os.getenv("OPENAI_API_KEY")

In [4]:
llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

In [5]:
from langchain.llms import openai
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [6]:
response_json={
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct_answer": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct_answer": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct_answer": "correct answer"
    }
}

In [7]:
Template='''
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple 
choice questions for {subject} in {tone} tone. Make sure that the questions are not repeated and check all
question to be confirming the text as well. Make sure to format your responses like {response_json} and use
it as a guide     
'''


quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "respone_json"],
    template=Template
)

In [8]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [9]:
Template2="""
You are an expert english grammarian and a writer. Given a multiple choice Quiz for {subject} students,
you need to evaluate  the complexity of the quiz and give a complete analysis of the quiz. Only use 50 words
at max. If the quiz is not at par the with the cognitive and the analytical ability of the student, update
the quiz questions and change the tone such that it perfectly fits the student's analytical ability.
Quiz_MCQ:
{quiz}


"""

In [10]:
quiz_evaluation_prompt=PromptTemplate(input_variables=['subject','quiz'], template=Template2)

In [11]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [12]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=['quiz', 'review'], verbose=True)

In [13]:
file_path='src\\data.txt'
with open(file_path, 'r') as file:
    text=file.read()

In [14]:
# Serialize the python dic into JSON-formatted string 
json.dumps(response_json)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct_answer": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct_answer": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct_answer": "correct answer"}}'

In [15]:
number =6
subject="Machine Learning"
tone="simple"

In [16]:
# Track token usage using get_openai_callback
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {   
            "text":text,
            "number":number,
            "subject":subject,
            "tone":tone,
            "response_json":json.dumps(response_json)
        }
)

c:\Users\avimi\.conda\envs\myenv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The near future will be dominated by several missions. Currently in development, are ESA's Euclid mission which will fly NASA furnished detectors and JAXA's XRISM (X-Ray Imaging and Spectroscopy) mission which uses NASA furnished technologies that will help provide breakthroughs in the study of structure formation of the universe, outflows from galaxy nuclei, and dark matter.

Completing the missions in development, supporting the operational missions, and funding the research and analysis programs will consume most of the Astrophysics Division resources.

In October 2021, NASA selected a new Explorer Mission, the gamma-ray telescope COSI (Compton Spectrometer and Imager). COSI will study the recent history of star birth, star death, and the formation of chemical elements in the Milky Way.

In March 2017, NASA selected the Explorer Mission of Opportunity GUSTO (Galactic/Extragala

In [19]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 1256
Prompt Tokens: 808
Completion Tokens: 448
Total Cost: 0.002108


In [40]:
quiz=response.get("quiz", None).strip()
quiz

'{"1": {"mcq": "What is the purpose of Physics?", "options": {"a": "To find the best restaurants in town", "b": "To study the interaction of matter and energy", "c": "To explore the history of ancient civilizations", "d": "To learn how to play musical instruments"}, "correct_answer": "b"},\n\n"2": {"mcq": "Which of the following is NOT a branch of Physics mentioned in the text?", "options": {"a": "Chemical Physics", "b": "Classical Physics", "c": "Modern Physics", "d": "Astrophysics"}, "correct_answer": "a"},\n\n"3": {"mcq": "What is the focus of Classical Physics?", "options": {"a": "Studying quantum mechanics", "b": "Exploring relativity theory", "c": "Understanding macroscopic phenomena", "d": "Investigating subatomic particles"}, "correct_answer": "c"},\n\n"4": {"mcq": "Which branch of Physics deals with the Nanoworld to the planets?", "options": {"a": "Astrophysics", "b": "Quantum Physics", "c": "Classical Physics", "d": "Modern Physics"}, "correct_answer": "d"},\n\n"5": {"mcq": "

In [17]:
response.get("review", None).strip()

'The quiz is complex for Machine Learning students as it focuses on specific details of NASA missions and astrophysics. To make it more suitable, simplify questions to focus on general concepts like the goals of space missions and key milestones in astrophysics.'

In [21]:
json.loads(quiz)

{'1': {'mcq': 'What is the purpose of Physics?',
  'options': {'a': 'To find the best restaurants in town',
   'b': 'To study the interaction of matter and energy',
   'c': 'To explore the history of ancient civilizations',
   'd': 'To learn how to play musical instruments'},
  'correct_answer': 'b'},
 '2': {'mcq': 'Which of the following is NOT a branch of Physics mentioned in the text?',
  'options': {'a': 'Chemical Physics',
   'b': 'Classical Physics',
   'c': 'Modern Physics',
   'd': 'Astrophysics'},
  'correct_answer': 'a'},
 '3': {'mcq': 'What is the focus of Classical Physics?',
  'options': {'a': 'Studying quantum mechanics',
   'b': 'Exploring relativity theory',
   'c': 'Understanding macroscopic phenomena',
   'd': 'Investigating subatomic particles'},
  'correct_answer': 'c'},
 '4': {'mcq': 'Which branch of Physics deals with the Nanoworld to the planets?',
  'options': {'a': 'Astrophysics',
   'b': 'Quantum Physics',
   'c': 'Classical Physics',
   'd': 'Modern Physics'}

In [22]:
quiz_table_data=[]
for key,value in json.loads(quiz).items():
    mcq=value['mcq']
    options="|".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value["options"].items()
        ]

    )

    correct=value['correct_answer']
    quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct":correct})

In [23]:
quiz_table_data

[{'MCQ': 'What is the purpose of Physics?',
  'Choices': 'a:To find the best restaurants in town|b:To study the interaction of matter and energy|c:To explore the history of ancient civilizations|d:To learn how to play musical instruments',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is NOT a branch of Physics mentioned in the text?',
  'Choices': 'a:Chemical Physics|b:Classical Physics|c:Modern Physics|d:Astrophysics',
  'Correct': 'a'},
 {'MCQ': 'What is the focus of Classical Physics?',
  'Choices': 'a:Studying quantum mechanics|b:Exploring relativity theory|c:Understanding macroscopic phenomena|d:Investigating subatomic particles',
  'Correct': 'c'},
 {'MCQ': 'Which branch of Physics deals with the Nanoworld to the planets?',
  'Choices': 'a:Astrophysics|b:Quantum Physics|c:Classical Physics|d:Modern Physics',
  'Correct': 'd'},
 {'MCQ': 'What is the foundation of Physics?',
  'Choices': 'a:Observations and experiments|b:Guessing and assuming|c:Intuition and opinion|d:Random

In [24]:
import pandas as pd
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,What is the purpose of Physics?,a:To find the best restaurants in town|b:To st...,b
1,Which of the following is NOT a branch of Phys...,a:Chemical Physics|b:Classical Physics|c:Moder...,a
2,What is the focus of Classical Physics?,a:Studying quantum mechanics|b:Exploring relat...,c
3,Which branch of Physics deals with the Nanowor...,a:Astrophysics|b:Quantum Physics|c:Classical P...,d
4,What is the foundation of Physics?,a:Observations and experiments|b:Guessing and ...,a
5,What is the role of mathematical analysis in P...,a:To confuse scientists|b:To provide a clearer...,b


In [61]:
def get_table_data(quiz_str):
    try:
        
        quiz_table_data=[]

        for key, value in quiz_str.items():
            mcq=value['mcq']
            options="||".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value["options"].items()
        ]

        )

        correct=value['correct_answer']
        quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct":correct})
    except Exception as e:
        traceback.print_exception(type(e),e,e.__traceback__)
        return False

In [66]:
data_table=get_table_data(quiz)

In [68]:
data_table

In [67]:
pd.DataFrame(data_table)

""


In [64]:
data_table

In [72]:
import traceback
def get_table_data(quiz_str):
    try:
        #quiz_dict=json.loads(quiz_str)
        quiz_table_data=[]

        for key, value in quiz_str.items():
            mcq=value['mcq']
            options="||".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value["options"].items()
        ]

        )
            correct=value['correct_answer']
            quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct":correct})
        return quiz_table_data
    except Exception as e:
        traceback.print_exception(type(e),e,e.__traceback__)
        return False

In [76]:
quiz={"1": {"mcq": "What is the purpose of Physics?", "options": {"a": "To find the quantitative physical laws", "b": "To study history", "c": "To explore the oceans", "d": "To learn about astronomy"}, "correct_answer": "a"},

"2": {"mcq": "How many major branches of Physics are there?", "options": {"a": "One", "b": "Two", "c": "Three", "d": "Four"}, "correct_answer": "b"},

"3": {"mcq": "Which branch of Physics deals with the interaction of matter and energy?", "options": {"a": "Biology", "b": "Chemistry", "c": "Physics", "d": "Modern Physics"}, "correct_answer": "c"}}

In [75]:
def get_table_data(quiz_str):
    try:
       
        quiz_table_data=[]

        for key, value in quiz_str.items():
            mcq=value['mcq']
            options="||".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value["options"].items()
        ]

        )
            correct=value['correct_answer']
            quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct":correct})
        return quiz_table_data
    except Exception as e:
        traceback.print_exception(type(e),e,e.__traceback__)
        return False

In [78]:
pd.DataFrame(get_table_data(quiz))

,MCQ,Choices,Correct
0,What is the purpose of Physics?,a:To find the quantitative physical laws||b:To...,a
1,How many major branches of Physics are there?,a:One||b:Two||c:Three||d:Four,b
2,Which branch of Physics deals with the interac...,a:Biology||b:Chemistry||c:Physics||d:Modern Ph...,c


In [59]:
for key, value in quiz.items():
            mcq=value['mcq']
            options="||".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value["options"].items()
        ]

        )
            correct=value['correct_answer']
            quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct":correct})

In [60]:
quiz_table_data

[{'MCQ': 'What is the purpose of Physics?',
  'Choices': 'a:To find the best restaurants in town||b:To study the interaction of matter and energy||c:To explore the history of ancient civilizations||d:To learn how to play musical instruments',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is NOT a branch of Physics mentioned in the text?',
  'Choices': 'a:Chemical Physics||b:Classical Physics||c:Modern Physics||d:Astrophysics',
  'Correct': 'a'},
 {'MCQ': 'What is the focus of Classical Physics?',
  'Choices': 'a:Studying quantum mechanics||b:Exploring relativity theory||c:Understanding macroscopic phenomena||d:Investigating subatomic particles',
  'Correct': 'c'},
 {'MCQ': 'Which branch of Physics deals with the Nanoworld to the planets?',
  'Choices': 'a:Astrophysics||b:Quantum Physics||c:Classical Physics||d:Modern Physics',
  'Correct': 'd'},
 {'MCQ': 'What is the foundation of Physics?',
  'Choices': 'a:Observations and experiments||b:Guessing and assuming||c:Intuition and op

In [2]:
q={"1": {"mcq": "What is the purpose of Physics?", "options": {"a": "To find the best recipes", "b": "To find the quantitative physical laws", "c": "To predict the weather", "d": "To study historical events"}, "correct_answer": "b"},

"2": {"mcq": "What are the two major branches of Physics mentioned in the text?", "options": {"a": "Chemistry and Biology", "b": "Classical Physics and Quantum Physics", "c": "Physics and Mathematics", "d": "Modern Physics and Ancient Physics"}, "correct_answer": "b"}, 

"3": {"mcq": "What is Physics commonly known as?", "options": {"a": "The study of plants", "b": "The study of matter and energy interaction", "c": "The study of history", "d": "The study of language"}, "correct_answer": "b"}}

In [3]:
def get_table_data(quiz_str):
    try:
       
        quiz_table_data=[]

        for key, value in quiz_str.items():
            mcq=value['mcq']
            options="||".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value["options"].items()
        ]

        )
            correct=value['correct_answer']
            quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct":correct})
        return quiz_table_data
    except Exception as e:
        traceback.print_exception(type(e),e,e.__traceback__)
        return False

In [4]:
get_table_data(q)

[{'MCQ': 'What is the purpose of Physics?',
  'Choices': 'a:To find the best recipes||b:To find the quantitative physical laws||c:To predict the weather||d:To study historical events',
  'Correct': 'b'},
 {'MCQ': 'What are the two major branches of Physics mentioned in the text?',
  'Choices': 'a:Chemistry and Biology||b:Classical Physics and Quantum Physics||c:Physics and Mathematics||d:Modern Physics and Ancient Physics',
  'Correct': 'b'},
 {'MCQ': 'What is Physics commonly known as?',
  'Choices': 'a:The study of plants||b:The study of matter and energy interaction||c:The study of history||d:The study of language',
  'Correct': 'b'}]